# **Functional 모델: 다중입력**

<center><img src = "https://github.com/Jangrae/img/blob/master/carseats2.png?raw=true" width=800/></center>

## **1. 환경준비**

### (1) 라이브러리 불러오기

In [ ]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

### (2) 함수 만들기

In [ ]:
# 함수 만들기
def dl_history_plot(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history['loss'], label='Train Loss', marker='.')
    plt.plot(history['val_loss'], label='Validation Loss', marker='.')

    plt.title('Learning Curve', size=15, pad=20)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()

### (3) 데이터 준비

In [ ]:
# 데이터 준비
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/Carseats.csv'
data = pd.read_csv(path)
data.drop(columns='Education', inplace=True)
data.head()

**데이터 설명**

- Sales: 각 지역 판매량(단위: 1,000개)
- CompPrice: 경쟁사 가격(단위: 달러)
- Income: 지역 평균 소득(단위: 1,000달러)
- Advertising: 각 지역, 회사의 광고 예산(단위: 1,000달러)
- Population: 지역 인구수(단위: 1,000명)
- Price: 자사 지역별 판매가격
- ShelveLoc: 진열상태
- Age: 지역 인구의 평균 연령
- Urban: 도심 지역 여부(Yes, No)
- US: 매장이 미국에 있는지 여부(Yes, No)

## **2. 데이터 전처리**

- 아래 코드는 다중 입력 모델을 연습하기 위해 임의로 변수를 구분한 것입니다.
- 성능과 상관없이 다중 입력 연습을 위한 것입니다.

### (1) 기본 전처리

In [ ]:
# x, y 분리
target = 'Sales'
x = data.drop(columns=target)
y = data.loc[:, target]

# 가변수화
cat_cols = ['ShelveLoc', 'US', 'Urban']
x = pd.get_dummies(x, columns=cat_cols, drop_first=True, dtype=int)

# 학습용, 검증용 분리
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1)

# 스케일링
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

# 테이터프레임으로 변환
x_train = pd.DataFrame(x_train, columns=x.columns)
x_val = pd.DataFrame(x_val, columns=x.columns)

In [ ]:
# 확인
x_train.head()

### (2) 입력 나누기

- 입력 1: 판매 관련 정보 - Advertising, Price, ShelveLoc, US, Urban, CompPrice
- 입력 2: 외부 환경 정보 - Income, Population, Age


In [ ]:
# 입력 1
in_col = ['Advertising', 'Price', 'CompPrice', 'ShelveLoc_Good', 'ShelveLoc_Medium', 'US_Yes', 'Urban_Yes']
x_train1 = x_train[in_col]
x_val1 = x_val[in_col]
x_train1.head()

In [ ]:
# 입력 2
x_train2 = x_train.drop(columns=in_col)
x_val2 = x_val.drop(columns=in_col)
x_train2.head()

## **3. 모델링**


### (1) 모델 선언

In [ ]:
# 크기 확인
print(x_train1.shape)
print(x_train2.shape)

In [ ]:
# 메모리 정리
clear_session()

# 입력
i1 = Input(shape=(7, ))
i2 = Input(shape=(3, ))

# 입력을 위한 Hidden Layer
h1_1 = Dense(10, activation='relu')(i1)
h1_2 = Dense(20, activation='relu')(i2)

# 두 Hidden Layer 결합
cl = concatenate([h1_1, h1_2])

# 추가 Hidden Layer
h2 = Dense(8, activation='relu')(cl)

# 출력 Layer
ol = Dense(1)(h2)

# 모델 선언
model = Model(inputs=[i1, i2], outputs=ol)

# 모델 요약
model.summary()

In [ ]:
# pydot 설치
#!pip install pydot

In [ ]:
# 모델 구조 시각화
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot

dot = model_to_dot(model, show_shapes=True, rankdir='TD')
dot.set_graph_defaults(dpi='60')
SVG(dot.create(prog='dot', format='svg'))

### (2) 모델 학습

In [ ]:
# 학습 설정
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

In [ ]:
# 모델 학습
hist = model.fit([x_train1, x_train2], y_train, epochs=50, validation_split=0.2, verbose=0).history

In [ ]:
# 학습 곡선
dl_history_plot(hist)

### (3) 예측 및 성능 평가

In [ ]:
# 예측
y_pred = model.predict([x_val1, x_val2])

# 성능 평가
print('* MAE:', mean_absolute_error(y_val, y_pred))
print('* R2 Score:', r2_score(y_val, y_pred))

## **5. 실습: 보스턴 집값 예측**

- 보스턴 집값 데이터를 이용해서 입력을 2개로 나누고 모델링을 수행합니다.
- 입력을 임의로 나눠도 좋고, 어떠한 관점을 가지고 나눠도 좋습니다.



### (1) 데이터 전처리

#### 1) 데이터 준비

In [ ]:
# 데이터 준비
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/boston.csv'
data = pd.read_csv(path)
data.drop(columns='black', inplace=True)
data.head()

**데이터 설명**

- crim: 자치시(Town)별 1인당 범죄율
- zn: 25,000 평방피트를 초과하는 거주지역 비율
- indus: 비소매상업지역이 점유하고 있는 토지 비율
- chas: 찰스강에 대한 더미 변수 (= 1 강 경계에 위치; 0 나머지)
- nox: 10ppm당 농축 일산화질소
- rm: 주택 1가구당 평균 방 개수
- age: 1940년 이전에 건축된 소유주택 비율
- dis: 5개 보스턴 직업센터까지 접근성 지수
- rad: 방사형 도로까지의 접근성 지수
- tax: 10,000달러 당 재산세율
- lstat: 모집단 하위 계층의 비율(%)
- medv: 본인 소유 주택가격(중앙값) (단위:$1,000)

#### 2) 기본 전처리

In [ ]:
# x, y 분리
target = 'medv'
x = data.drop(columns=target)
y = data.loc[:, target]

# 학습용, 검증용 분리
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1)

# 스케일링
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

# 데이터프레임으로 변환
x_train = pd.DataFrame(x_train, columns=x.columns)
x_val = pd.DataFrame(x_val, columns=x.columns)

In [ ]:
# 확인
x_train.head()

#### 3) 입력 나누기

In [ ]:
# 입력 1:
in_col = [ ]

x_train1 = x_train[in_col]
x_val1 = x_val[in_col]

# 확인
x_train1.head()

In [ ]:
# 입력 2:
x_train2 = x_train.drop(columns=in_col)
x_val2 = x_val.drop(columns=in_col)

# 확인
x_train2.head()

In [ ]:
# 크기 확인
print(x_train1.shape)
print(x_train2.shape)

### (2) 모델링

#### 1) 모델 선언

In [ ]:
# 메모리 정리
clear_session()

# 입력
i1 =
i2 =

# 첫 번째 입력을 위한 Hidden Layer
h1_1 =
h1_2 =

# 두 Hidden Layer 결합
cl =

# 추가 Hidden Layer
h2 =

# 출력 Layer
ol =

# 모델 선언
model = Model(inputs=[i1, i2], outputs=ol)

# 모델 요약
model.summary()

In [ ]:
# 모델 구조 시각화
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot

dot = model_to_dot(model, show_shapes=True)
dot.set_graph_defaults(dpi='60')
SVG(dot.create(prog='dot', format='svg'))

#### 2) 모델 학습

In [ ]:
# 학습 설정
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

In [ ]:
# 모델 학습
hist = model.fit([x_train1, x_train2], y_train, epochs=20, validation_split=0.2, verbose=0).history

In [ ]:
# 학습 곡선
dl_history_plot(hist)

#### 3) 예측 및 성능 평가

In [ ]:
# 예측
y_pred = model.predict([x_val1, x_val2])

# 성능 평가
print('* MAE:', mean_absolute_error(y_val, y_pred))
print('* R2 Score:', r2_score(y_val, y_pred))